In [36]:
from search import Problem
import random

class VacuumProblem(Problem):
    
    def __init__(self):
        #initial_state = ([random.randrange(0,2) for i in range(0,9)], random.randrange(0,9))
        
        initial_state = [[1,1,1,0,0,0,0,0,0], 4]
        goal_state = [ 0 for i in range(0,9)]
        super().__init__(initial_state, goal_state)
          
    def actions(self, state):
        loc = state[1]
        if loc == 0:
            return ["DOWN", "RIGHT", "SUCK"]
        elif loc == 3:
            return ["UP", "DOWN", "RIGHT", "SUCK"]
        elif loc == 6:
            return ["UP", "RIGHT", "SUCK"]
        elif loc == 1:
            return ["DOWN", "LEFT", "RIGHT", "SUCK"]
        elif loc == 4:
            return ["UP", "DOWN", "LEFT", "RIGHT", "SUCK"]
        elif loc == 7:
            return ["UP", "LEFT", "RIGHT", "SUCK"]
        elif loc == 2:
            return ["DOWN", "LEFT", "SUCK"]
        elif loc == 5:
            return ["UP", "DOWN", "LEFT", "SUCK"]
        elif loc == 8:
            return ["UP", "LEFT", "SUCK"]
        else:
            return "FAILURE"
        
    def result(self, state, action):
        if action is "SUCK":
            state[0][state[1]] = 0
            return state
        
        elif action is "LEFT":
            state[1] -= 1
            return state
        
        elif action is "RIGHT":
            state[1] += 1
            return state
        
        elif action is "UP":
            state[1] = state[1] - 3
            return state
        
        elif action is "DOWN":
            state[1] = state[1] + 3
            return state 
            
    
    def path_cost(self, cur_cost, state1, action, state2):
        if action is "SUCK":
            print("Path Cost:", cur_cost + abs(-1 + (-2 * state2.count(1))))
            return cur_cost + abs(-1 + (-2 * state2.count(1)))
        else:
            print("Path Cost:", cur_cost + abs(-1 + (-2 * state1.count(1))))
            return cur_cost + abs(-1 + (-2 * state1.count(1)))
        
    def h(self, n):
        h_cost = 0
        min_dist = 10000000
        num_actions = 0
        dirty = 0
        pos = n.state[1]
        count = 0
        for s in n.state[0]:
            if s == 1:
                dirty += 1
                dist = abs(pos - count)
                if min_dist > dist:
                    min_dist = dist
            count += 1
        h_cost = dirty + min_dist
        print("H cost:", h_cost)    
        return h_cost


In [37]:
from search import Node

class VacuumNode(Node):
    
    def __init__(self, state, parent=None, action=None, path_cost=0,):
        super().__init__(state, parent, action, path_cost)
        
        
    def child_node(self, problem, action):
        
        next = problem.result(self.state, action)
        return VacuumNode(state=next, parent=self, action=action,
                    path_cost=problem.path_cost(self.path_cost, self.state,
                                      action, next))
    
    def __str__(self):
        return "{} {} {} {}".format(self.state, self.action, self.path_cost)

        
        
        


In [38]:
from search import *

def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    
   
    f = memoize(f, 'f')
    node = VacuumNode(problem.initial)
    
    if problem.goal_test(node.state):
        return(node)
    
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    
    explored = {}
    while frontier:
        node = frontier.pop()
        
        if problem.goal_test(node.state):
            return(node)
        
        explored.update({tuple(node.state[0]):1})
        for child in node.expand(problem):
            if explored[tuple(child.state[0])] != 1 and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
    return None            

def astar_search(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    
    h = memoize(h or problem.h, 'h')
    node = best_first_graph_search(problem, lambda n: n.path_cost + h(n))
    return(node)

VP = VacuumProblem()
print(astar_search(VP))

H cost: 5
Path Cost: 3
Path Cost: 1
Path Cost: 1
Path Cost: 1
Path Cost: 1
None
